In [1]:
import os
import torch
import sys

sys.path.append('I:/mmdeploy/build/bin/Release')
os.add_dll_directory('D:/opencv/build/x64/vc16/bin')
os.add_dll_directory('D:/onnxruntime-win-x64-1.8.1/lib')

<AddedDllDirectory('D:/onnxruntime-win-x64-1.8.1/lib')>

In [2]:
from mmdeploy_python import Detector, PoseDetector

# hand pose

In [3]:
detector = Detector('hand2_yolov8n', 'cpu', 0)
pose_detector = PoseDetector('hand', 'cpu', 0)

In [ ]:
import cv2

def draw_labels(frame, bbox):
        [left, top, right, bottom] = bbox[0:4].astype(int)
        # coco数据集标签对应的颜色
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 1)

In [53]:
img = cv2.imread('test.jpg')

bboxes, label, _ = detector(img)

[left, top, right, bottom] = bboxes[0][0:4].astype(int)


result = pose_detector(img, bboxes[..., :4])

scores = result[..., 2]
keypoints = (result[..., :2]).astype(int)

print(keypoints)

skeleton = [(1, 0), (1, 2), (2, 3), (3, 4), (5, 0), 
            (5, 6), (6, 7), (7, 8), (12, 0), 
            (9, 10), (10, 11), (11, 12), (16, 0), 
            (13, 14), (14, 15), (15, 16), (20, 0), 
            (17, 18), (18, 19), (19, 20)]



for kpts, score in zip(keypoints, scores):

  for (u,v) in skeleton:
    cv2.line(img, (kpts[u][0], kpts[u][1]), (kpts[v][0], kpts[v][1]), (0, 255, 0), 1, cv2.LINE_AA)

  for kpt in range(len(kpts)):
    if kpt == 0:
      cv2.circle(img, kpts[kpt], 1, (255, 0, 0), 2, cv2.LINE_AA)
    else:
      cv2.circle(img, kpts[kpt], 1, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imwrite('pose.jpg', img)

[[[407 146]
  [400 237]
  [398 218]
  [410 194]
  [405 168]
  [264 245]
  [297 223]
  [324 206]
  [355 180]
  [235 237]
  [266 211]
  [297 187]
  [333 163]
  [228 211]
  [259 192]
  [292 170]
  [324 151]
  [232 175]
  [261 166]
  [280 158]
  [319 146]]]


True

# whole body

In [54]:
detector = Detector('yolov8', 'cpu', 0)
pose_detector = PoseDetector('wholebody', 'cpu', 0)

In [79]:
img = cv2.imread('whole_body.jpg')

bboxes, label, _ = detector(img)

# print(bboxes[1, :4], label)

# [left, top, right, bottom] = bboxes[0][0:4].astype(int)


result = pose_detector(img, bboxes[0, :4])
print(result)
scores = result[..., 2]
keypoints = (result[..., :2]).astype(int)

for  kpts, score in zip(keypoints, scores):
    for kpt in range(len(kpts)):
        if kpt >=23 and kpt <= 95:
          cv2.circle(img, kpts[kpt], 1, (255, 0, 0), 8, cv2.LINE_AA)
        elif kpt >=0 and kpt <=16:
          cv2.circle(img, kpts[kpt], 1, (0, 255, 0), 4, cv2.LINE_AA)
        elif kpt >=95 and kpt <= 132:
          cv2.circle(img, kpts[kpt], 1, (0, 255, 255), 4, cv2.LINE_AA)
        else:
          cv2.circle(img, kpts[kpt], 1, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imwrite('whole_body_pose.jpg', img)

# 设置body 17关键点的连线关系
body_skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
                    (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
                    (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
body_palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
                (255, 153, 255), (153, 204, 255), (255, 102, 255),
                (255, 51, 255), (102, 178, 255),
                (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
                (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
                (0, 0, 255), (255, 0, 0), (255, 255, 255)]
link_color = [0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16]
point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]


# for kpts, score in zip(keypoints, scores):

#   for (u,v) in skeleton:
#     cv2.line(img, (kpts[u][0], kpts[u][1]), (kpts[v][0], kpts[v][1]), (0, 255, 0), 1, cv2.LINE_AA)

#   for kpt in range(len(kpts)):
#     if kpt == 0:
#       cv2.circle(img, kpts[kpt], 1, (255, 0, 0), 2, cv2.LINE_AA)
#     else:
#       cv2.circle(img, kpts[kpt], 1, (0, 0, 255), 2, cv2.LINE_AA)

# cv2.imwrite('pose.jpg', img)

[[[3.6950000e+02 3.1843658e+02 9.2455971e-01]
  [4.1984015e+02 2.6821039e+02 9.8286009e-01]
  [3.2922784e+02 2.6821039e+02 9.2325783e-01]
  [4.7018024e+02 2.9834613e+02 8.8402206e-01]
  [2.7888776e+02 2.8830084e+02 9.0133852e-01]
  [5.4065643e+02 5.2938678e+02 7.5761986e-01]
  [1.7820746e+02 5.5952252e+02 8.2526952e-01]
  [6.1113263e+02 7.2024664e+02 8.3814931e-01]
  [1.2786731e+02 8.5083478e+02 8.0235690e-01]
  [5.5072443e+02 4.2893439e+02 8.0443603e-01]
  [1.6813947e+02 1.0919207e+03 8.3667374e-01]
  [4.9031635e+02 1.0517397e+03 5.1438600e-01]
  [2.5875165e+02 1.0517397e+03 5.5920708e-01]
  [4.0977216e+02 2.1798413e+02 7.2731513e-01]
  [4.3997614e+02 3.4857233e+02 3.5212710e-01]
  [2.8895575e+02 2.1798413e+02 6.9916391e-01]
  [3.3929590e+02 2.8830084e+02 6.6211450e-01]
  [4.1984015e+02 2.8830084e+02 1.4853011e+00]
  [4.1984015e+02 2.8830084e+02 1.5093488e+00]
  [2.8895575e+02 2.1798413e+02 6.6879267e-01]
  [2.7888776e+02 2.3807465e+02 1.0614179e+00]
  [2.7888776e+02 2.3807465e+02 9.9

True